In [1]:
import pickle

To make the `pickle` readable, we must add translation to the `sys.path`:

In [2]:
import sys
sys.path.append(r'../')
sys.path.append(r'../translation/')

In [3]:
import translation

In [4]:
filepath = "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according "\
           "to Shah_20211103_1556_20_ribosomes_3600s.p"

In [5]:
with open(filepath, 'rb') as f:
  results = pickle.load(f)

In [6]:
results.keys()

dict_keys(['proteome', 'peptide_bonds', 'transcriptome', 'timerange', 'timecourses', 'description', 'time_stamp', 'n_ribosomes', 'n_tRNA', 'duration'])